### Задача №1.
Используя данные о хоккеистах в `hockey_players.csv`, проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие: а) роста, б) BMI.

__Решение:__

Значимость отличия роста и BMI хоккеистов из Финляндии, Норвегии и Дании проверим с помощью однофакторного дисперсионного анализа, используя F-критерий Фишера. За нулевую гипотезу принимаем утверждение об отсутствии отличия роста и BMI хоккеистов из Финляндии, Норвегии и Дании.

Для начала загрузим данные, избавимся от дубликатов и выделим рассматриваемые признаки, разобив на выборки по значению категории, в нашем случае по странам "FIN", "NOR", "DEN":

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('hockey_players.csv')
df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


In [3]:
df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

In [4]:
df = df.drop_duplicates(subset=['name', 'birth'], keep='last')

In [5]:
heights = [df.loc[df['country'] == country, 'height'] for country in ['FIN', 'NOR', 'DEN']]
bmis = [df.loc[df['country'] == country, 'bmi'] for country in ['FIN', 'NOR', 'DEN']]

Используемая статистика представляет из себя отношение дисперсии между уровнями к дисперсии внутри уровней:

$$\normalsize F = \dfrac{\sigma_b^2}{\sigma_w^2},$$

Дисперсии находятся по формулам:

$$\normalsize \sigma_b^2 = \dfrac{SS_b}{k - 1}, \:\: \sigma_w^2 = \dfrac{SS_w}{n - k},$$

где $n$ - размер выборки, $k$ - количество уровней (уникальных значений категориального признака).

$SS_b, SS_w$ в свою очередь вычисляются следующим образом:

$$\normalsize SS_b = \displaystyle\sum_{i=1}^k \left( \overline{Y_i} - \overline{Y} \right)^2 n_i, \:\:
SS_w = \displaystyle\sum_{i=1}^k \displaystyle\sum_{j=1}^{n_i} \left( y_{ij} - \overline{Y_i} \right)^2,$$
где $\overline{Y}$ - общее среднее выборочное, $\overline{Y_i}$ - среднее выборочное $i$-й выборки, $y_{ij}$ — $j$-й элемент $i$-й выборки, $n_i$ - размер $i$-й выборки.

Напишем функцию и подсчитаем F-статистики для роста и BMI:

In [6]:
def f_stat(ys):
    y = np.concatenate(ys)

    ss_b = sum((yi.mean() - y.mean()) ** 2 * yi.size for yi in ys)
    ss_w = sum(((yi - yi.mean()) ** 2).sum() for yi in ys)
    
    sigma_b = ss_b / (len(ys) - 1)
    sigma_w = ss_w / (y.size - len(ys))
    
    return sigma_b / sigma_w

In [7]:
f_stat_height = f_stat(heights)
f_stat_bmi = f_stat(bmis)

f_stat_height, f_stat_bmi

(3.8026212813131024, 5.918577060155501)

В предположении верности нулевой гипотезы $F$-статистика имеет распределение Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$. Критическая область здесь правосторонняя:
$$\normalsize\Omega_\alpha = \left( t_{1 - \alpha, \: k_1, k_2}, \infty \right),$$
где $t_{\alpha, \: k_1, k_2}$ — квантиль порядка $\alpha$ для распределения Фишера с параметрами $k_1$, $k_2$.

Для обоих признаков (рост, bmi) критическая область будет одинаковой, так как рассматривается одна и та же выборка.   
Зададим уровень значимости $\alpha=0.05$ и подсчитаем квантиль порядка $1-\alpha$ распределения Фишера с параметрами для нашей выборки: 

In [8]:
from scipy import stats

In [9]:
k = len(bmis)
n = bmis[0].size + bmis[1].size + bmis[2].size
k1 = k - 1
k2 = n - k
alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
t

3.016700272452327

Критическая область: $\normalsize\Omega_\alpha = \left(3.017, \infty \right)$. И значения $F$-статистики для роста и BMI, равные $3.8026$ и $5.9186$ соответственно, в нее попадают. Нулевая гипотеза отвергается, следовательно __является значимым__ отличие в росте и BMI хоккеистов из Финляндии, Норвегии и Дании.

__Ответ:__   
__Является значимым__ отличие в росте и BMI хоккеистов из Финляндии, Норвегии и Дании.